In [1]:
# FOR HDI: http://ec2-54-174-131-205.compute-1.amazonaws.com/API/Information.php
# Human development classification
# HDI classifications are based on HDI fixed cutoff points, which are derived from the quartiles of dis
# tributions of the component indicators. The cutoffpoints are HDI of less than 0.550 for low human development,
#  0.550–0.699 for medium human development, 0.700–0.799 for high human development and 0.800 or greater 
#  for very high human development.
# Codes:
# 44206 - Total Population (millions)
# 137506 Human Development Index (HDI)

In [2]:
import json
from turtle import back
import requests

In [3]:
url_HDRO = "http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/indicator_id=137506,44206/year=2019"
url_USGS = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson"

response_HDRO = requests.get(url_HDRO)
print(f"HDRO: {response_HDRO}")
response_USGS = requests.get(url_USGS)
print(f"USGS: {response_USGS}")

HDRO: <Response [200]>
USGS: <Response [200]>


In [4]:
data_HDRO = response_HDRO.json()
# print(json.dumps(data_HDRO, indent=4, sort_keys=True))
data_USGS = response_USGS.json()

In [5]:
import pandas as pd

countryCodes_df = pd.read_csv("../../data/all.csv")

In [6]:
# countryCodes_df
clean_cc_df = countryCodes_df.copy()[["name", "alpha-2", "alpha-3"]]
clean_cc_df['alpha-2'][153] = 'NA'
clean_cc_df['alpha-2'][153]

'NA'

In [7]:
clean_cc_df.to_csv("clean_cc_df.csv")

In [8]:
df_HDRO = pd.read_json(url_HDRO)
df_HDRO = df_HDRO.reset_index()
df_HDRO

,index,indicator_value,country_name,indicator_name
0,AFG,"{'44206': {'2019': 38.042}, '137506': {'2019':...",Afghanistan,NaN
1,AGO,"{'44206': {'2019': 31.825}, '137506': {'2019':...",Angola,NaN
2,ALB,"{'44206': {'2019': 2.8810000000000002}, '13750...",Albania,NaN
3,AND,"{'44206': {'2019': 0.077}, '137506': {'2019': ...",Andorra,NaN
4,ARE,"{'44206': {'2019': 9.771}, '137506': {'2019': ...",United Arab Emirates,NaN
...,...,...,...,...
192,ZAF,"{'44206': {'2019': 58.558}, '137506': {'2019':...",South Africa,NaN
193,ZMB,"{'44206': {'2019': 17.861}, '137506': {'2019':...",Zambia,NaN
194,ZWE,"{'44206': {'2019': 14.645}, '137506': {'2019':...",Zimbabwe,NaN
195,44206,NaN,NaN,Total population (millions)


In [9]:
clean_HDRO_df = pd.DataFrame(columns=['CountryName', 'CountryCode', 'HDI', "TotalPopulation"]) 
# clean_HDRO_df
i=1
for index, country in df_HDRO.iterrows():
    try:
        value_totalPopulation = float(country["indicator_value"]['44206']['2019'])
        value_HDI = float(country["indicator_value"]['137506']['2019'])
        clean_HDRO_df = clean_HDRO_df.append([{"CountryName": country["country_name"], "CountryCode": country["index"], "HDI": value_HDI, "TotalPopulation": value_totalPopulation}], ignore_index=True)
    except:
        print(country["index"], country["indicator_value"])

clean_HDRO_df


MCO {'44206': {'2019': 0.039}}
NRU {'44206': {'2019': 0.011}}
PRK {'44206': {'2019': 25.666}}
SMR {'44206': {'2019': 0.034}}
SOM {'44206': {'2019': 15.443}}
TUV {'44206': {'2019': 0.012}}
44206 nan
137506 nan


,CountryName,CountryCode,HDI,TotalPopulation
0,Afghanistan,AFG,0.511,38.042
1,Angola,AGO,0.581,31.825
2,Albania,ALB,0.795,2.881
3,Andorra,AND,0.868,0.077
4,United Arab Emirates,ARE,0.890,9.771
...,...,...,...,...
184,Samoa,WSM,0.715,0.197
185,Yemen,YEM,0.470,29.162
186,South Africa,ZAF,0.709,58.558
187,Zambia,ZMB,0.584,17.861


In [10]:
clean_HDRO_df.to_csv("clean_HDRO_df.csv")

In [11]:
clean_HDRO_df = clean_HDRO_df.merge(clean_cc_df, left_on="CountryCode", right_on="alpha-3", how="inner")
clean_HDRO_df =clean_HDRO_df.drop(["CountryCode"], axis=1)


In [12]:
clean_HDRO_df

,CountryName,HDI,TotalPopulation,name,alpha-2,alpha-3
0,Afghanistan,0.511,38.042,Afghanistan,AF,AFG
1,Angola,0.581,31.825,Angola,AO,AGO
2,Albania,0.795,2.881,Albania,AL,ALB
3,Andorra,0.868,0.077,Andorra,AD,AND
4,United Arab Emirates,0.890,9.771,United Arab Emirates,AE,ARE
...,...,...,...,...,...,...
184,Samoa,0.715,0.197,Samoa,WS,WSM
185,Yemen,0.470,29.162,Yemen,YE,YEM
186,South Africa,0.709,58.558,South Africa,ZA,ZAF
187,Zambia,0.584,17.861,Zambia,ZM,ZMB


In [13]:
clean_HDRO_df.to_csv("clean_HDRO_df_MERGED.csv")

In [14]:
for index, row in clean_HDRO_df.iterrows():
    print(row)

CountryName        Afghanistan
HDI                      0.511
TotalPopulation         38.042
name               Afghanistan
alpha-2                     AF
alpha-3                    AFG
Name: 0, dtype: object
CountryName        Angola
HDI                 0.581
TotalPopulation    31.825
name               Angola
alpha-2                AO
alpha-3               AGO
Name: 1, dtype: object
CountryName        Albania
HDI                  0.795
TotalPopulation      2.881
name               Albania
alpha-2                 AL
alpha-3                ALB
Name: 2, dtype: object
CountryName        Andorra
HDI                  0.868
TotalPopulation      0.077
name               Andorra
alpha-2                 AD
alpha-3                AND
Name: 3, dtype: object
CountryName        United Arab Emirates
HDI                                0.89
TotalPopulation                   9.771
name               United Arab Emirates
alpha-2                              AE
alpha-3                             ARE
Na

In [15]:
# Import SQL Alchemy
from sqlalchemy import create_engine

# Import module to create relationship patterns
from sqlalchemy.orm import relationship

# Import and establish Base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Table, Column, Integer, String, Float, BigInteger, ForeignKey

In [16]:
# Create classes:
# Countries & Earthquakes

class Countries(Base):  #Parent
    __tablename__ = 'countries'
    # id = Column(Integer, primary_key=True)
    CountryCode = Column(String(255), primary_key=True)
    CountryName = Column(String(255))
    Population = Column(Float)  # Millions
    HDIndex = Column(Float) # Index
    earthquake = relationship("Earthquakes")

class Earthquakes(Base):    #Child
    __tablename__ = 'earthquakes'
    # id = Column(Integer, primary_key=True)
    EarthquakeID = Column(String(255), primary_key=True)
    Latitude = Column(Float)
    Longitude = Column(Float)
    Depth = Column(Float)
    DateTime = Column(BigInteger)
    # CountryCode = Column(String(255))   # Foreign Key?
    country_id = Column(String(255), ForeignKey('countries.CountryCode'))
    # parent = relationship("Countries", back_populates="children")


# print(data_USGS["features"])

In [17]:
import reverse_geocoder as rg
from time import sleep
from tqdm.auto import tqdm

In [18]:
# Create Database Connection
# ---------------------------
# Establish Connection
engine = create_engine("sqlite:///project2.sqlite")
conn = engine.connect()

# Create Countries and Earthquakes tables within the database
Base.metadata.create_all(conn)

# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [19]:
for index, country in tqdm(clean_HDRO_df.iterrows(), total=len(clean_HDRO_df)):
    # sleep(0.1)
    try:
        new_country = Countries(CountryCode=country["alpha-2"],CountryName=country["CountryName"],Population=country["TotalPopulation"],HDIndex=country["HDI"])
        session.add(new_country)
        session.commit()
    except:
        print(country["alpha-2"], country["CountryName"], country["TotalPopulation"], country["HDI"])


100%|██████████| 189/189 [00:22<00:00,  8.46it/s]


In [20]:
# i=1
# Create specific instances of the Countries and Earthquakes classes
for earthquake in tqdm(data_USGS["features"], total=len(data_USGS["features"])):
    # sleep(0.1)

    # if i<=10:
    # print(earthquake["id"], earthquake["geometry"]["coordinates"][0], earthquake["geometry"]["coordinates"][1], earthquake["geometry"]["coordinates"][2], earthquake["properties"]["time"])
    try:
        lon = earthquake["geometry"]["coordinates"][0]
        lat = earthquake["geometry"]["coordinates"][1]
        depth1 = earthquake["geometry"]["coordinates"][2]
        ctry_code = rg.search((lat,lon))[0]["cc"]
        new_earthquake = Earthquakes(EarthquakeID=earthquake["id"], Longitude=lon, Latitude=lat, Depth=depth1, DateTime=earthquake["properties"]["time"], country_id=ctry_code)

        # print(f"{i}, {earthquake['id']}, {lon}, {lat}, {depth1}, {earthquake['properties']['time']}, {ctry_code}")
        # Add new_earthquake to the current session
        session.add(new_earthquake)
        # Add new_country to the current session

        # Commit objects to the database
        session.commit()
        # print(i)
        # i = i+1
        # else:
        #     break
    except:
        print(f"{earthquake['id']}, {lon}, {lat}, {depth1}, {earthquake['properties']['time']}, {ctry_code}")


conn.close()
engine.dispose()


print("DATABASE COMPLETED")

# earthquake_list = session.query(Earthquakes)
# for earthquake in earthquake_list:
#     print(earthquake.Longitude)

  0%|          | 0/9052 [00:00<?, ?it/s]

Loading formatted geocoded file...


  1%|          | 85/9052 [02:44<3:36:48,  1.45s/it]

pr2022019002, -68.3258, 18.0678, 69, 1642561506380, US


  1%|          | 100/9052 [03:09<3:55:07,  1.58s/it]